In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup

import time

from datetime import datetime, timedelta
from dateutil import parser

import pandas as pd

## NBCNews

In [2]:
options = Options()
#options.headless = True
b = webdriver.Chrome(options=options)

url = 'https://www.nbcnews.com/search/?q=gaza+war'
b.get(url)

In [3]:
dropdown = b.find_element(By.CLASS_NAME, 'gsc-selected-option-container')
dropdown.click()

time.sleep(1)

date_option = b.find_element(By.XPATH, '/html/body/div[@id="content"]/div[@id="___gcse_0"]/div[@class="gsc-control-cse gsc-control-cse-en"]/div[@class="gsc-control-wrapper-cse"]/div[@class="gsc-results-wrapper-nooverlay gsc-results-wrapper-visible"]/div[@class="gsc-above-wrapper-area"]/table/tbody/tr/td/div/div/div[@class="gsc-option-menu"]/div[@class="gsc-option-menu-item"][1]')
date_option.click()

In [4]:
response_text = b.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div/div[5]/div[2]/div/div/div[2]/div').get_attribute("outerHTML")

soup = BeautifulSoup(response_text, 'html.parser')

# Find the specific elements that contain the article links
div_list = soup.find_all('div', class_='gsc-cursor-page')
last_page_number = int(div_list[-1].text)
last_page_number

10

In [5]:
# sample input:
# '2 hours ago ... Bombs went off and lights went out as a blackout plunged Gaza into chaos. Israeli forces have begun what Prime Minister Benjamin Netanyahu\xa0...'
# from
# div_list[0].find('div', class_='gs-bidi-start-align gs-snippet').text

def convert_to_local_time(soup_find_string):
    
    time_string = soup_find_string.split("...")[0].rstrip()
    time_strings_list = time_string.split()
    duration_str = time_strings_list[1]
    
    if (time_string == 'just now'):
        time_stamp = datetime.now()
    elif (duration_str == 'second') or (duration_str == 'seconds'):
        delta_num = int(time_strings_list[0])
        time_stamp = datetime.now() - timedelta(seconds=delta_num)
    elif (duration_str == 'minute') or (duration_str == 'minutes') or (duration_str == 'mins') or (duration_str == 'min'):
        delta_num = int(time_strings_list[0])
        time_stamp = datetime.now() - timedelta(minutes=delta_num)
    elif (duration_str == 'hour') or (duration_str == 'hours'):
        delta_num = int(time_strings_list[0])
        time_stamp = datetime.now() - timedelta(hours=delta_num)
    elif (duration_str == 'day') or (duration_str == 'days'):
        delta_num = int(time_strings_list[0])
        time_stamp = datetime.now() - timedelta(days=delta_num)
    elif (duration_str == 'month') or (duration_str == 'months'):
        # FOXNews mark Oct 14 as 1 month ago in a search result on Oct 30
        delta_num = int(time_strings_list[0]) * 30
        time_stamp = datetime.now() - timedelta(days=delta_num)
    else:
        date_format = '%b %d, %Y'
        time_stamp = datetime.strptime(time_string, date_format)
    
    return time_stamp

In [6]:
date_list = []
url_list = []
title_list = []

next_page_number = 1

while next_page_number <= last_page_number:
    
    response_text = b.find_element(By.CLASS_NAME, 'gsc-expansionArea').get_attribute("outerHTML")
    soup = BeautifulSoup(response_text, 'html.parser')

    # Find the specific elements that contain the article links
    div_list = soup.find_all('div', class_='gs-webResult gs-result')

    for result in div_list:
        news_description = result.find('div', class_='gs-bidi-start-align gs-snippet').text
        article_date = convert_to_local_time(news_description)
        date_list.append(article_date)

        article_url = result.find('a', class_='gs-title')['href']
        url_list.append(article_url)

        article_title = result.find('a', class_='gs-title').text
        title_list.append(article_title)
    
    
    # get the page number of the next page
    current_page_str = b.find_element(By.CLASS_NAME, 'gsc-cursor-current-page').text
    next_page_number = int(current_page_str) + 1
    next_page_number
    
    if next_page_number > last_page_number:
        break

    # click the next page link
    next_page_button = b.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div/div[5]/div[2]/div/div/div[2]/div/div[{}]'.format(next_page_number))
    next_page_button.click()
    time.sleep(1)


d = {'datetime': date_list, 'url': url_list, 'title':title_list}
df_NBCNews_articles = pd.DataFrame(data=d)

In [7]:
df_NBCNews_articles

,datetime,url,title
0,2023-10-28 21:22:28.503212,https://www.nbcnews.com/news/world/israel-hama...,Israel-Gaza conflict could spread into Middle ...
1,2023-10-30 17:22:28.503295,https://www.nbcnews.com/news/world/live-blog/i...,Israel-Hamas war live updates: Fears rise over...
2,2023-10-29 21:22:28.503362,https://www.nbcnews.com/nightly-news/video/isr...,Israel-Hamas war enters new phase as Israel ex...
3,2023-10-30 00:22:28.503487,https://www.nbcnews.com/news/world/live-blog/i...,Thousands break into Gaza aid warehouse
4,2023-10-28 21:22:28.503709,https://www.nbcnews.com/news/world/live-blog/i...,Airstrikes pound Gaza as Israel expands ground...
...,...,...,...
105,2023-10-19 00:00:00.000000,https://www.today.com/parents/mothers-protect-...,Two Mothers Share How They're Protecting Their...
106,2023-10-19 00:00:00.000000,https://www.msnbc.com/alex-wagner-tonight/watc...,Release of two American hostages by Hamas offe...
107,2023-10-19 00:00:00.000000,https://www.nbcnews.com/news/world/live-blog/i...,Israel-Hamas war live updates: 2 hostages rele...
108,2023-10-19 00:00:00.000000,https://www.msnbc.com/all-in/watch/gaza-condit...,"Gaza conditions worsen amid food, water, fuel ..."


In [8]:
# Finally, you can retrieve the loaded content
nbcnews_page_source = b.page_source

# Close the browser
b.quit()

## FOXNews

In [9]:
options = Options()
#options.headless = True
b = webdriver.Chrome(options=options)

url = 'https://www.foxnews.com/search-results/search?q=gaza%20war'
b.get(url)

In [10]:
# set date range from Oct 1, 2023

dropdown_mm = b.find_element(By.XPATH, '//*[@id="wrapper"]/div[2]/div[1]/div/div[2]/div[3]/div[1]/div[1]/button')
dropdown_mm.click()

time.sleep(1)

dropdown_mm10 = b.find_element(By.XPATH, '//*[@id="10"]')
dropdown_mm10.click()

time.sleep(1)

dropdown_dd = b.find_element(By.XPATH, '//*[@id="wrapper"]/div[2]/div[1]/div/div[2]/div[3]/div[1]/div[2]/button')
dropdown_dd.click()

time.sleep(1)

dropdown_dd01 = b.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/div[1]/div/div[2]/div[3]/div[1]/div[2]/ul/li[1]')
dropdown_dd01.click()

time.sleep(1)

dropdown_YYYY = b.find_element(By.XPATH, '//*[@id="wrapper"]/div[2]/div[1]/div/div[2]/div[3]/div[1]/div[3]/button')
dropdown_YYYY.click()

time.sleep(1)

dropdown_YYYY2023 = b.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/div[1]/div/div[2]/div[3]/div[1]/div[3]/ul/li[1]')
dropdown_YYYY2023.click()

time.sleep(1)


# click search
button_search = b.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/div[1]/div/div[1]/div[2]/div')
button_search.click()


# load all available search results (limited to 100 by FOXNews)
button_load_more = b.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/div[2]/div/div[3]/div[2]')
class_button_load = ''
time.sleep(1)

while class_button_load != 'button load-more hide':
    button_load_more.click()
    time.sleep(1)
    class_button_load = b.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/div[2]/div/div[3]/div[2]').get_attribute("class")

In [11]:
date_list = []
url_list = []
title_list = []


response_text = b.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/div[2]/div/div[3]/div[1]').get_attribute("outerHTML")
soup = BeautifulSoup(response_text, 'html.parser')

# Find the specific elements that contain the article links
div_list = soup.find_all('article', class_='article')

for result in div_list:
    news_description = result.find('span', class_='time').text
    article_date = convert_to_local_time(news_description)
    date_list.append(article_date)

    article_url = result.find('a')['href']
    url_list.append(article_url)

    article_title = result.find('h2', class_='title').text
    title_list.append(article_title)


d = {'datetime': date_list, 'url': url_list, 'title':title_list}
df_FOXNews_articles = pd.DataFrame(data=d)

In [12]:
df_FOXNews_articles

,datetime,url,title
0,2023-10-27 21:23:02.676207,https://www.foxnews.com/live-news/october-27-i...,Israel launches second ground incursion into G...
1,2023-10-26 21:23:02.676252,https://www.foxnews.com/live-news/israel-hamas...,Israel conducts ground incursion in Gaza tanks...
2,2023-10-30 17:23:02.676287,https://www.foxnews.com/live-news/october-30-i...,Israeli military pushes deeper into Gaza as ID...
3,2023-10-29 21:23:02.676322,https://www.foxnews.com/video/6340044924112,WATCH LIVE: Live look at Israel-Gaza border as...
4,2023-10-28 21:23:02.676355,https://www.foxnews.com/video/6340093735112,WATCH LIVE: Live look at Israel-Gaza border as...
...,...,...,...
95,2023-09-30 21:23:02.679187,https://www.foxnews.com/politics/kirby-us-acti...,Kirby: US 'actively trying' to find hostages i...
96,2023-09-30 21:23:02.679212,https://www.foxnews.com/world/un-refugee-agenc...,UN refugee agency: Gaza becoming a 'hellhole' ...
97,2023-10-23 21:23:02.679237,https://www.foxnews.com/politics/kirby-says-us...,Kirby says US asking Israel 'tough questions' ...
98,2023-09-30 21:23:02.679263,https://www.foxnews.com/us/nyc-protest-rally-p...,NYC protest rally for Palestinians blames Isra...


In [13]:
# Finally, you can retrieve the loaded content
foxnews_page_source = b.page_source

# Close the browser
b.quit()

## Reuters

In [15]:
options = Options()
#options.headless = True
b = webdriver.Chrome(options=options)

url = 'https://www.reuters.com/search/news?blob=gaza+war&sortBy=date&dateRange=all'
b.get(url)

load_more_button = b.find_element(By.CLASS_NAME, 'search-result-more-txt')

In [16]:
# b.find_element(By.CLASS_NAME, 'search-result-list').get_attribute("outerHTML")

In [17]:
response_text = b.find_element(By.CLASS_NAME, 'search-result-list').get_attribute("outerHTML")

soup = BeautifulSoup(response_text, 'html.parser')

# Find the specific elements that contain the article links
div_list = soup.find_all('div', class_='search-result-indiv')

# for result in div_list:
#     article_url = result.find('a')['href']
#     print("Article URL:", article_url)

oldes_time_string = div_list[-1].find('h5').text
oldest_article_time = parser.parse(oldes_time_string)

/Users/zhounanlin/anaconda3/envs/anlp/lib/python3.11/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [18]:
while oldest_article_time > datetime(2023, 10, 1, 0, 0):
    load_more_button.click()
    time.sleep(1)
    
    response_text = b.find_element(By.CLASS_NAME, 'search-result-list').get_attribute("outerHTML")

    soup = BeautifulSoup(response_text, 'html.parser')

    # Find the specific elements that contain the article links
    div_list = soup.find_all('div', class_='search-result-indiv')

    # for result in div_list:
    #     article_url = result.find('a')['href']
    #     print("Article URL:", article_url)

    oldest_time_string = div_list[-1].find('h5').text
    oldest_article_time = parser.parse(oldest_time_string)

In [19]:
# Finally, you can retrieve the loaded content
reuters_page_source = b.page_source

# Close the browser
b.quit()

In [20]:
# save the following info to a dataframe

date_list = []
url_list = []
title_list = []

for result in div_list:
    
    article_date_string = result.find('h5').text
    article_date = parser.parse(article_date_string)
    date_list.append(article_date)
    
    article_url = result.find('a')['href']
    url_list.append(article_url)
    
    article_title = result.find('h3', class_='search-result-title').text
    title_list.append(article_title)
    
#     print(article_date)
#     print(article_url)
#     print(article_title)

d = {'datetime': date_list, 'url': url_list, 'title':title_list}
df_reuters_articles = pd.DataFrame(data=d)

In [21]:
df_reuters_articles

,datetime,url,title
0,2023-10-30 23:27:00,/article/idUSKBN31V03Z,Gold on track for biggest monthly rise in 11 o...
1,2023-10-30 23:27:00,/article/idUSL1N3C1031,PRECIOUS-Gold on track for biggest monthly ris...
2,2023-10-30 20:26:00,/article/idUSL1N3C100H,Oil rises ahead of key central bank meetings a...
3,2023-10-30 18:58:00,/article/idUSKBN31U14C,"US officials, Jewish leaders meet to address a..."
4,2023-10-30 17:47:00,/article/idUSKBN31U0IG,Graphic pro-Israel ads make their way into chi...
...,...,...,...
1015,2023-10-04 08:34:00,/article/idUSKBN31410C,Gaza unrest sends message about economic miser...
1016,2023-09-27 11:01:00,/article/idUSKBN30X1FE,"UN peace envoy, Egypt working to restore calm ..."
1017,2023-09-24 06:14:00,/article/idUSL1N3B001W,UPDATE 2-Israeli forces kill two Palestinians ...
1018,2023-09-22 14:54:00,/article/idUSKBN30S1KD,Israeli drone hits Gaza as violent protests rage


## Stack and export to csv

In [2]:
# df_NBCNews_articles = pd.read_csv('backup_news_df_231031/df_NBCNews_articles.csv')
# df_FOXNews_articles = pd.read_csv('backup_news_df_231031/df_FOXNews_articles.csv')
# df_reuters_articles = pd.read_csv('backup_news_df_231031/df_reuters_articles.csv')

In [6]:
df_NBCNews_articles['owner_username'] = 'NBCNews'
df_FOXNews_articles['owner_username'] = 'FoxNews'
df_reuters_articles['owner_username'] = 'Reuters'

,Unnamed: 0,datetime,url,title,owner_username
0,0,2023-10-28 21:22:28.503212,https://www.nbcnews.com/news/world/israel-hama...,Israel-Gaza conflict could spread into Middle ...,NBCNews
1,1,2023-10-30 17:22:28.503295,https://www.nbcnews.com/news/world/live-blog/i...,Israel-Hamas war live updates: Fears rise over...,NBCNews
2,2,2023-10-29 21:22:28.503362,https://www.nbcnews.com/nightly-news/video/isr...,Israel-Hamas war enters new phase as Israel ex...,NBCNews
3,3,2023-10-30 00:22:28.503487,https://www.nbcnews.com/news/world/live-blog/i...,Thousands break into Gaza aid warehouse,NBCNews
4,4,2023-10-28 21:22:28.503709,https://www.nbcnews.com/news/world/live-blog/i...,Airstrikes pound Gaza as Israel expands ground...,NBCNews
...,...,...,...,...,...
105,105,2023-10-19 00:00:00.000000,https://www.today.com/parents/mothers-protect-...,Two Mothers Share How They're Protecting Their...,NBCNews
106,106,2023-10-19 00:00:00.000000,https://www.msnbc.com/alex-wagner-tonight/watc...,Release of two American hostages by Hamas offe...,NBCNews
107,107,2023-10-19 00:00:00.000000,https://www.nbcnews.com/news/world/live-blog/i...,Israel-Hamas war live updates: 2 hostages rele...,NBCNews
108,108,2023-10-19 00:00:00.000000,https://www.msnbc.com/all-in/watch/gaza-condit...,"Gaza conditions worsen amid food, water, fuel ...",NBCNews


In [11]:
stacked_news_dfs = pd.DataFrame()
stacked_news_dfs = pd.concat([stacked_news_dfs, df_NBCNews_articles, df_FOXNews_articles, df_reuters_articles], ignore_index=True)
stacked_news_dfs

,Unnamed: 0,datetime,url,title,owner_username
0,0,2023-10-28 21:22:28.503212,https://www.nbcnews.com/news/world/israel-hama...,Israel-Gaza conflict could spread into Middle ...,NBCNews
1,1,2023-10-30 17:22:28.503295,https://www.nbcnews.com/news/world/live-blog/i...,Israel-Hamas war live updates: Fears rise over...,NBCNews
2,2,2023-10-29 21:22:28.503362,https://www.nbcnews.com/nightly-news/video/isr...,Israel-Hamas war enters new phase as Israel ex...,NBCNews
3,3,2023-10-30 00:22:28.503487,https://www.nbcnews.com/news/world/live-blog/i...,Thousands break into Gaza aid warehouse,NBCNews
4,4,2023-10-28 21:22:28.503709,https://www.nbcnews.com/news/world/live-blog/i...,Airstrikes pound Gaza as Israel expands ground...,NBCNews
...,...,...,...,...,...
1225,1015,2023-10-04 08:34:00,/article/idUSKBN31410C,Gaza unrest sends message about economic miser...,Reuters
1226,1016,2023-09-27 11:01:00,/article/idUSKBN30X1FE,"UN peace envoy, Egypt working to restore calm ...",Reuters
1227,1017,2023-09-24 06:14:00,/article/idUSL1N3B001W,UPDATE 2-Israeli forces kill two Palestinians ...,Reuters
1228,1018,2023-09-22 14:54:00,/article/idUSKBN30S1KD,Israeli drone hits Gaza as violent protests rage,Reuters


In [13]:
stacked_news_dfs.to_csv('stacked_news_df.csv', index=False)